In [1]:
from grafos import Accion, Estado, Problema, Nodo

Crear métodos auxiliares

In [2]:
def crear_nodo_raiz(problema):
    estado_raiz = problema.estado_inicial
    acciones_raiz = {}
    if estado_raiz.nombre in problema.acciones.keys():
        acciones_raiz = problema.acciones[estado_raiz.nombre]
    raiz = Nodo(estado_raiz, None, acciones_raiz, None)
    raiz.costo = 0
    raiz.heuristicas = problema.heuristicas[estado_raiz.nombre]
    raiz.valores = dict(raiz.heuristicas.items())
    return raiz

def crear_nodo_hijo(problema, padre, accion, agregar = True):
    nuevo_estado = problema.resultado(padre.estado, accion)
    acciones_nuevo = {}
    if nuevo_estado.nombre in problema.acciones.keys():
        acciones_nuevo = problema.acciones[nuevo_estado.nombre]
    hijo = Nodo(nuevo_estado, accion, acciones_nuevo, padre)
    costo = padre.costo
    costo += problema.costo_accion(padre.estado, accion)
    hijo.costo = costo
    hijo.heuristicas = problema.heuristicas[hijo.estado.nombre]
    hijo.valores = {estado: heuristica + hijo.costo for estado, heuristica in hijo.heuristicas.items()}
    if agregar:
        padre.hijos.append(hijo)
    return hijo

def sacar_siguiente(frontera, metrica="valor", criterio="menor", objetivos = None):
    mejor = frontera[0]
    for nodo in frontera[1:]:
        for objetivo in objetivos:
            if metrica == "valor":
                valor_nodo = nodo.valores[objetivo.nombre]
                valor_mejor = mejor.valores[objetivo.nombre]
                if (criterio == "menor" and valor_nodo < valor_mejor):
                    mejor = nodo
                elif (criterio == "mayor" and valor_nodo > valor_mejor):
                    mejor = nodo
            elif metrica == "heuristica":
                heuristica_nodo = nodo.heuristicas[objetivo.nombre]
                heuristica_mejor = mejor.heuristicas[objetivo.nombre]
                if (criterio == "menor" and heuristica_nodo < heuristica_mejor):
                    mejor = nodo
                elif (criterio == "mayor" and heuristica_nodo > heuristica_mejor):
                    mejor = nodo
            if metrica == "costo":
                if (criterio == "menor" and nodo.costo_camino < mejor.costo_camino):
                    mejor = nodo
                elif (criterio == "mayor" and nodo.costo_camino > mejor.costo_camino):
                    mejor = nodo
    frontera.remove(mejor)
    return mejor

def muestra_solucion(objetivo = None, problema_resolver = None):
    if not objetivo:
        print("No hay solución")
        return
    nodo = objetivo
    while nodo:
        estado = nodo.estado.nombre
        valores = [nodo.valores[objetivo.nombre]
                   for objetivo
                   in problema_resolver.estado_objetivo]
        valor = min(valores)
        msg = "Estado {0}, Valor {1}"
        print(msg.format(estado, valor))
        costo_total = nodo.costo
        msg = "  Costo: {0}"
        print(msg.format(costo_total))
        heuristicas_objetivos = [nodo.heuristicas[objetivo.nombre]
                                 for objetivo
                                 in problema_resolver.estado_objetivo]
        heuristica = min(heuristicas_objetivos)
        msg = "  Heurística: {0}"
        print(msg.format(heuristica))
        if nodo.accion:
            accion = nodo.accion.nombre
            padre = nodo.padre.estado
            costo = problema_resolver.costo_accion(padre, nodo.accion)
            if accion:
                msg = "<--- {0} [{1}] ---"
                print(msg.format(accion, costo))
        nodo = nodo.padre

Crear la función que ejecuta el problema

In [3]:
def voraz(problema):
    raiz = crear_nodo_raiz(problema)
    frontera = [raiz, ]
    explorados = set()
    while True:
        if not frontera:
            return None
        nodo = sacar_siguiente(frontera, 'heuristica',
                               objetivos=problema.estado_objetivo)
        if problema.es_objetivo(nodo.estado):
            return nodo
        explorados.add(nodo.estado)
        if not nodo.acciones:
            continue
        for nombre_accion in nodo.acciones.keys():
            accion = Accion(nombre_accion)
            hijo = crear_nodo_hijo(problema, nodo, accion)
            estados_frontera = [nodo.estado for nodo in frontera]
            if hijo.estado in explorados or hijo.estado in estados_frontera:
                buscar = [nodo for nodo in frontera
                          if nodo.estado == hijo.estado]
                if buscar:
                    heuristica_hijo = [hijo.heuristicas[objetivo.nombre]
                                      for objetivo
                                      in problema.estado_objetivo]
                    heuristica_buscar = [buscar[0].heuristicas[objetivo.nombre]
                                        for objetivo
                                        in problema.estado_objetivo]
                    minimo_hijo = min(heuristica_hijo)
                    minimo_buscar = min(heuristica_buscar)
                    if minimo_hijo < minimo_buscar:
                        indice = frontera.index(buscar[0])
                        frontera[indice] = hijo
            else:
                frontera.append(hijo)

Crear la estructura de datos del problema

In [4]:
if __name__ == '__main__':
    accN = Accion('N')
    accS = Accion('S')
    accE = Accion('E')
    accO = Accion('O')
    accNE = Accion('NE')
    accNO = Accion('NO')
    accSE = Accion('SE')
    accSO = Accion('SO')

    lanoi = Estado('Lanoi', [accNE])
    nahoi = Estado('Nahoi', [accSO, accNO, accNE])
    ruun = Estado('Ruun', [accNO, accNE, accE, accSE])
    milos = Estado('Milos', [accO, accSO, accN])
    ghildo = Estado('Ghildo', [accN, accE, accSE])
    kuart = Estado('Kuart', [accO, accSO, accNE])
    boomon = Estado('Boomon', [accN, accSO])
    goorum = Estado('Goorum', [accO, accS])
    shiphos = Estado('Shiphos', [accO, accE])
    nokshos = Estado('Nokshos', [accNO, accS, accE])
    pharis = Estado('Pharis', [accNO, accSO])
    khamin = Estado('Khamin', [accSE, accNO, accO])
    tarios = Estado('Tarios', [accO, accNO, accNE, accE])
    peranna = Estado('Peranna', [accO, accE])
    khandon = Estado('Khandon', [accO, accS])
    tawa = Estado('Tawa', [accSO, accSE, accNE])
    theer = Estado('Theer', [accSO, accSE])
    roria = Estado('Roria', [accNO, accSO, accE])
    kosos = Estado('Kosos', [accO])

    acciones = {'Lanoi':{'NE':nahoi},
                'Nahoi':{'SO':lanoi, 'NO':ruun, 'NE':milos},
                'Ruun':{'NO':ghildo, 'NE':kuart, 'E':milos, 'SE':nahoi },
                'Milos':{'O':ruun, 'SO':nahoi, 'N':khandon},
                'Ghildo':{'N':nokshos, 'E':kuart, 'SE':ruun},
                'Kuart':{'O':ghildo, 'SO':ruun, 'NE':boomon},
                'Boomon':{'N':goorum, 'SO':kuart},
                'Goorum':{'O':shiphos, 'S':boomon},
                'Shiphos':{'O':nokshos, 'E':goorum},
                'Nokshos':{'NO':pharis, 'S':ghildo, 'E':shiphos},
                'Pharis':{'NO':khamin, 'SO':nokshos},
                'Khamin':{'SE':pharis, 'NO':tawa, 'NE':roria},
                'Tarios':{'O':khamin, 'NO':tawa, 'NE':roria, 'E':peranna},
                'Peranna':{'O':tarios, 'E':khandon},
                'Khandon':{'O':peranna, 'S':milos},
                'Tawa':{'SO':khamin, 'SE':tarios, 'NE':theer},
                'Theer':{'SO':tawa, 'SE':roria},
                'Roria':{'NO':theer, 'SO':tarios, 'E':kosos},
                'Kosos':{'O':roria}}
    
    costos = {'Lanoi':{'NE':42},
                'Nahoi':{'SO':42, 'NO':21, 'NE':95},
                'Ruun':{'NO':88, 'NE':16, 'E':90, 'SE':21 },
                'Milos':{'O':90, 'SO':95, 'N':133},
                'Ghildo':{'N':17, 'E':92, 'SE':88},
                'Kuart':{'O':92, 'SO':16, 'NE':83},
                'Boomon':{'N':8, 'SO':83},
                'Goorum':{'O':59, 'S':8},
                'Shiphos':{'O':71, 'E':59},
                'Nokshos':{'NO':5, 'S':17, 'E':71},
                'Pharis':{'NO':29, 'SO':5},
                'Khamin':{'SE':29, 'NO':121, 'NE':98},
                'Tarios':{'O':98, 'NO':83, 'NE':57, 'E':82},
                'Peranna':{'O':82, 'E':44},
                'Khandon':{'O':44, 'S':133},
                'Tawa':{'SO':121, 'SE':83, 'NE':11},
                'Theer':{'SO':11, 'SE':36},
                'Roria':{'NO':36, 'SO':57, 'E':104},
                'Kosos':{'O':104}}
    
    heuristicas = {'Lanoi': {'Lanoi': 0, 'Nahoi': 32, 'Ruun': 43, 'Milos': 90, 'Ghildo': 81, 'Kuart': 50, 'Boomon': 90, 'Goorum': 95, 'Shiphos': 84,  'Nokshos': 108,
                             'Pharis': 111, 'Khamin': 124, 'Tarios': 145, 'Peranna': 157, 'Khandon': 182, 'Tawa': 180, 'Theer': 183, 'Roria': 174, 'Kosos': 224},
                   'Nahoi': {'Lanoi': 32, 'Nahoi': 0, 'Ruun': 12, 'Milos': 72, 'Ghildo': 73, 'Kuart': 18, 'Boomon': 57, 'Goorum': 63, 'Shiphos': 56, 'Nokshos': 93,
                             'Pharis': 97, 'Khamin': 107, 'Tarios': 117, 'Peranna': 125, 'Khandon': 150, 'Tawa': 153, 'Theer': 154, 'Roria': 143, 'Kosos': 191},
                   'Ruun': {'Lanoi': 43, 'Nahoi': 12, 'Ruun': 0, 'Milos': 71, 'Ghildo': 75, 'Kuart': 6,
                            'Boomon': 45,
                            'Goorum': 50,
                            'Shiphos': 45,
                            'Nokshos': 87,
                            'Pharis': 89,
                            'Khamin': 100,
                            'Tarios': 105,
                            'Peranna': 103,
                            'Khandon': 140,
                            'Tawa': 140,
                            'Theer': 142,
                            'Roria': 131,
                            'Kosos': 179},
                   'Milos': {'Lanoi': 90,
                             'Nahoi': 72,
                             'Ruun': 71,
                             'Milos': 0,
                             'Ghildo': 145,
                             'Kuart': 70,
                             'Boomon': 76,
                             'Goorum': 81,
                             'Shiphos': 103,
                             'Nokshos': 156,
                             'Pharis': 157,
                             'Khamin': 165,
                             'Tarios': 138,
                             'Peranna': 116,
                             'Khandon': 122,
                             'Tawa': 176,
                             'Theer': 174,
                             'Roria': 159,
                             'Kosos': 172},
                   'Ghildo': {'Lanoi': 81,
                              'Nahoi': 73,
                              'Ruun': 75,
                              'Milos': 145,
                              'Ghildo': 0,
                              'Kuart': 75,
                              'Boomon': 94,
                              'Goorum': 93,
                              'Shiphos': 61,
                              'Nokshos': 34,
                              'Pharis': 37,
                              'Khamin': 53,
                              'Tarios': 112,
                              'Peranna': 150,
                              'Khandon': 187,
                              'Tawa': 134,
                              'Theer': 140,
                              'Roria': 136,
                              'Kosos': 211},
                   'Kuart': {'Lanoi': 50,
                             'Nahoi': 18,
                             'Ruun': 6,
                             'Milos': 70,
                             'Ghildo': 75,
                             'Kuart': 0,
                             'Boomon': 38,
                             'Goorum': 43,
                             'Shiphos': 42,
                             'Nokshos': 87,
                             'Pharis': 89,
                             'Khamin': 98,
                             'Tarios': 98,
                             'Peranna': 107,
                             'Khandon': 134,
                             'Tawa': 135,
                             'Theer': 137,
                             'Roria': 124,
                             'Kosos': 173},
                   'Boomon': {'Lanoi': 90,
                              'Nahoi': 57,
                              'Ruun': 45,
                              'Milos': 76,
                              'Ghildo': 94,
                              'Kuart': 38,
                              'Boomon': 0,
                              'Goorum': 6,
                              'Shiphos': 36,
                              'Nokshos': 91,
                              'Pharis': 91,
                              'Khamin': 95,
                              'Tarios': 67,
                              'Peranna': 68,
                              'Khandon': 98,
                              'Tawa': 105,
                              'Theer': 104,
                              'Roria': 91,
                              'Kosos': 134},
                   'Goorum': {'Lanoi': 95,
                              'Nahoi': 63,
                              'Ruun': 50,
                              'Milos': 81,
                              'Ghildo': 93,
                              'Kuart': 43,
                              'Boomon': 6,
                              'Goorum': 0,
                              'Shiphos': 33,
                              'Nokshos': 88,
                              'Pharis': 87,
                              'Khamin': 92,
                              'Tarios': 62,
                              'Peranna': 64,
                              'Khandon': 97,
                              'Tawa': 98,
                              'Theer': 98,
                              'Roria': 85,
                              'Kosos': 132},
                   'Shiphos': {'Lanoi': 84,
                               'Nahoi': 56,
                               'Ruun': 45,
                               'Milos': 103,
                               'Ghildo': 61,
                               'Kuart': 42,
                               'Boomon': 36,
                               'Goorum': 33,
                               'Shiphos': 0,
                               'Nokshos': 55,
                               'Pharis': 55,
                               'Khamin': 63,
                               'Tarios': 64,
                               'Peranna': 92,
                               'Khandon': 127,
                               'Tawa': 97,
                               'Theer': 101,
                               'Roria': 92,
                               'Kosos': 156},
                   'Nokshos': {'Lanoi': 108,
                               'Nahoi': 93,
                               'Ruun': 87,
                               'Milos': 156,
                               'Ghildo': 34,
                               'Kuart': 87,
                               'Boomon': 91,
                               'Goorum': 88,
                               'Shiphos': 55,
                               'Nokshos': 0,
                               'Pharis': 3,
                               'Khamin': 18,
                               'Tarios': 86,
                               'Peranna': 133,
                               'Khandon': 171,
                               'Tawa': 103,
                               'Theer': 109,
                               'Roria': 109,
                               'Kosos': 189},
                   'Pharis': {'Lanoi': 111,
                              'Nahoi': 97,
                              'Ruun': 89,
                              'Milos': 157,
                              'Ghildo': 37,
                              'Kuart': 89,
                              'Boomon': 91,
                              'Goorum': 87,
                              'Shiphos': 55,
                              'Nokshos': 3,
                              'Pharis': 0,
                              'Khamin': 14,
                              'Tarios': 83,
                              'Peranna': 132,
                              'Khandon': 170,
                              'Tawa': 100,
                              'Theer': 107,
                              'Roria': 105,
                              'Kosos': 186},
                   'Khamin': {'Lanoi': 124,
                              'Nahoi': 107,
                              'Ruun': 100,
                              'Milos': 165,
                              'Ghildo': 53,
                              'Kuart': 98,
                              'Boomon': 95,
                              'Goorum': 92,
                              'Shiphos': 63,
                              'Nokshos': 18,
                              'Pharis': 14,
                              'Khamin': 0,
                              'Tarios': 77,
                              'Peranna': 128,
                              'Khandon': 168,
                              'Tawa': 89,
                              'Theer': 97,
                              'Roria': 97,
                              'Kosos': 182},
                   'Tarios': {'Lanoi': 145,
                              'Nahoi': 117,
                              'Ruun': 105,
                              'Milos': 138,
                              'Ghildo': 112,
                              'Kuart': 98,
                              'Boomon': 67,
                              'Goorum': 62,
                              'Shiphos': 64,
                              'Nokshos': 86,
                              'Pharis': 83,
                              'Khamin': 77,
                              'Tarios': 0,
                              'Peranna': 55,
                              'Khandon': 94,
                              'Tawa': 38,
                              'Theer': 38,
                              'Roria': 27,
                              'Kosos': 104},
                   'Peranna': {'Lanoi': 157,
                               'Nahoi': 125,
                               'Ruun': 103,
                               'Milos': 116,
                               'Ghildo': 150,
                               'Kuart': 107,
                               'Boomon': 68,
                               'Goorum': 64,
                               'Shiphos': 92,
                               'Nokshos': 133,
                               'Pharis': 132,
                               'Khamin': 128,
                               'Tarios': 55,
                               'Peranna': 0,
                               'Khandon': 38,
                               'Tawa': 80,
                               'Theer': 73,
                               'Roria': 59,
                               'Kosos': 66},
                   'Khandon': {'Lanoi': 182,
                               'Nahoi': 150,
                               'Ruun': 140,
                               'Milos': 122,
                               'Ghildo': 187,
                               'Kuart': 134,
                               'Boomon': 98,
                               'Goorum': 97,
                               'Shiphos': 127,
                               'Nokshos': 171,
                               'Pharis': 170,
                               'Khamin': 168,
                               'Tarios': 94,
                               'Peranna': 38,
                               'Khandon': 0,
                               'Tawa': 113,
                               'Theer': 107,
                               'Roria': 92,
                               'Kosos': 52},
                   'Tawa': {'Lanoi': 180,
                            'Nahoi': 153,
                            'Ruun': 140,
                            'Milos': 176,
                            'Ghildo': 134,
                            'Kuart': 135,
                            'Boomon': 105,
                            'Goorum': 98,
                            'Shiphos': 97,
                            'Nokshos': 103,
                            'Pharis': 100,
                            'Khamin': 89,
                            'Tarios': 38,
                            'Peranna': 80,
                            'Khandon': 113,
                            'Tawa': 0,
                            'Theer': 9,
                            'Roria': 21,
                            'Kosos': 105},
                   'Theer': {'Lanoi': 183,
                             'Nahoi': 154,
                             'Ruun': 142,
                             'Milos': 174,
                             'Ghildo': 140,
                             'Kuart': 137,
                             'Boomon': 104,
                             'Goorum': 98,
                             'Shiphos': 101,
                             'Nokshos': 109,
                             'Pharis': 107,
                             'Khamin': 97,
                             'Tarios': 38,
                             'Peranna': 73,
                             'Khandon': 107,
                             'Tawa': 9,
                             'Theer': 0,
                             'Roria': 16,
                             'Kosos': 96},
                   'Roria': {'Lanoi': 174,
                             'Nahoi': 143,
                             'Ruun': 131,
                             'Milos': 159,
                             'Ghildo': 136,
                             'Kuart': 124,
                             'Boomon': 91,
                             'Goorum': 85,
                             'Shiphos': 92,
                             'Nokshos': 109,
                             'Pharis': 105,
                             'Khamin': 97,
                             'Tarios': 27,
                             'Peranna': 59,
                             'Khandon': 92,
                             'Tawa': 21,
                             'Theer': 16,
                             'Roria': 0,
                             'Kosos': 87},
                   'Kosos': {'Lanoi': 224,
                             'Nahoi': 191,
                             'Ruun': 179,
                             'Milos': 172,
                             'Ghildo': 211,
                             'Kuart': 173,
                             'Boomon': 134,
                             'Goorum': 132,
                             'Shiphos': 156,
                             'Nokshos': 189,
                             'Pharis': 186,
                             'Khamin': 182,
                             'Tarios': 104,
                             'Peranna': 66,
                             'Khandon': 52,
                             'Tawa': 105,
                             'Theer': 96,
                             'Roria': 87,
                             'Kosos': 0}}
    
    objetivo_1 = [kosos]
    problema_1 = Problema(lanoi, objetivo_1, acciones, costos, heuristicas)
    problema_resolver = problema_1
    solucion = voraz(problema_resolver)
    muestra_solucion(solucion, problema_resolver)


Estado Kosos, Valor 557
  Costo: 557
  Heurística: 0
<--- E [104] ---
Estado Roria, Valor 540
  Costo: 453
  Heurística: 87
<--- NE [57] ---
Estado Tarios, Valor 500
  Costo: 396
  Heurística: 104
<--- O [82] ---
Estado Peranna, Valor 380
  Costo: 314
  Heurística: 66
<--- O [44] ---
Estado Khandon, Valor 322
  Costo: 270
  Heurística: 52
<--- N [133] ---
Estado Milos, Valor 309
  Costo: 137
  Heurística: 172
<--- NE [95] ---
Estado Nahoi, Valor 233
  Costo: 42
  Heurística: 191
<--- NE [42] ---
Estado Lanoi, Valor 224
  Costo: 0
  Heurística: 224
